<a href="https://colab.research.google.com/github/nasibehmohammadi/Thesis/blob/main/mini%20batch%20learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1️⃣ بارگذاری دیتاست شخصی (لطفاً نام فایل را تغییر بده!)
file_path = "/content/d1.csv"  # نام فایل خود را اینجا بگذار
df = pd.read_csv(file_path)

# 2️⃣ بررسی ساختار داده‌ها
print("🔍 بررسی ۵ سطر اول دیتاست:")
print(df.head())

# 3️⃣ مشخص کردن ویژگی‌های ورودی و خروجی
# نام ستون‌ها را بر اساس دیتاست خود تغییر بده!
feature_columns = ["B","C","D","E","F","G","H","I","J"]  # نام ویژگی‌ها
target_column = "A"  # نام ستون هدف (برچسب‌ها)

# تبدیل دیتافریم به آرایه NumPy
X_data = df[feature_columns].values
y_data = df[target_column].values

# مشخص کردن تعداد ویژگی‌ها
num_features = X_data.shape[1]

# 4️⃣ تنظیم داده‌ها برای مدل LSTM
time_steps = 10  # تعداد مراحل زمانی (مثلاً ۱۰ ورود اخیر هر کاربر)

# ایجاد دنباله‌های زمانی برای LSTM
X_sequences = []
y_sequences = []

for i in range(len(X_data) - time_steps):
    X_sequences.append(X_data[i : i + time_steps])  # داده‌های ۱۰ زمان اخیر
    y_sequences.append(y_data[i + time_steps])  # مقدار هدف بعد از ۱۰ مرحله

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# 5️⃣ تقسیم داده‌ها به مجموعه آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# 6️⃣ مقیاس‌بندی داده‌ها
scaler = MinMaxScaler()
X_train = X_train.reshape(-1, num_features)
X_test = X_test.reshape(-1, num_features)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(-1, time_steps, num_features)
X_test = X_test.reshape(-1, time_steps, num_features)

# 7️⃣ ایجاد مدل LSTM
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 8️⃣ کامپایل مدل
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 9️⃣ آموزش مدل روی دیتای شخصی
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
model.save("model.h5")
# 🔟 ارزیابی مدل
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ دقت مدل روی داده‌های تست: {accuracy * 100:.2f}%")

# ۱۱️⃣ پیش‌بینی روی یک نمونه جدید
test_sample = X_test[0].reshape(1, time_steps, num_features)
prediction = model.predict(test_sample)

if prediction > 0.5:
    print("🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.")
else:
    print("✅ ورود ایمن است.")


🔍 بررسی ۵ سطر اول دیتاست:
   A      B      C       D       E       F       G       H       I       J
0  1  39353  85475  117961  118300  123472  117905  117906  290919  117908
1  1  17183   1540  117961  118343  123125  118536  118536  308574  118539
2  1  36724  14457  118219  118220  117884  117879  267952   19721  117880
3  1  36135   5396  117961  118343  119993  118321  240983  290919  118322
4  1  42680   5905  117929  117930  119569  119323  123932   19793  119325
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


819/819 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - accuracy: 0.9355 - loss: 0.2609 - val_accuracy: 0.9435 - val_loss: 0.2174
Epoch 2/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - accuracy: 0.9405 - loss: 0.2279 - val_accuracy: 0.9435 - val_loss: 0.2204
Epoch 3/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9417 - loss: 0.2257 - val_accuracy: 0.9435 - val_loss: 0.2175
Epoch 4/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.9411 - loss: 0.2270 - val_accuracy: 0.9435 - val_loss: 0.2189
Epoch 5/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.9428 - loss: 0.2218 - val_accuracy: 0.9435 - val_loss: 0.2176
Epoch 6/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.9394 - loss: 0.2305 - val_accuracy: 0.9435 - val_loss: 0.2175


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9431 - loss: 0.2188
✅ دقت مدل روی داده‌های تست: 94.35%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.


In [17]:
import random
import random
import pandas as pd


In [18]:
# Load dataset
file_path = "/content/d1.csv"
df = pd.read_csv("/content/d1.csv")  # مسیر فایل داخل "" قرار داده شده است


# Generate 10 random access requests
random_access_requests = []
num_rows, num_cols = df.shape

for _ in range(10):
    row_idx = random.randint(0, num_rows - 1)  # انتخاب یک ردیف تصادفی
    random_request = df.iloc[row_idx].to_dict()  # تبدیل ردیف به دیکشنری
    random_access_requests.append(random_request)

# تبدیل به دیتافریم و نمایش
random_access_requests_df = pd.DataFrame(random_access_requests)
print(random_access_requests_df)  # نمایش خروجی در کنسول



   A       B      C       D       E       F       G       H       I       J
0  1   36424  53506  117916  118150  117941  118568  139079   19721  118570
1  1   18278   4468  117961  118413  119968  118321  117906  290919  118322
2  0   33230  56663  117902  117903  118507  118028  118368  117887  118030
3  1  111351  16830  117961  118327  119830  120773  118959  118960  120774
4  1   74555   4576  117961  118327  118391  117905  117906  290919  117908
5  1   20364   2908  117961  118343  118979  120344  303717  118424  120346
6  1   42093  52105  118595  118596   81476  121469  130834  118424  121471
7  1   80473   7243  117961  118343  118609  120357  125779  118424  120359
8  1   73547   4707  117961  118300  121108  119928  150810  118331  119929
9  1   40230  46149  117961  118327    4674  118784  174486  290919  118786


In [19]:
# Append the 10 random access requests to the dataset
df_updated = pd.concat([df, random_access_requests_df], ignore_index=True)

# Save the updated dataset for further processing
updated_file_path = "/content/updated_d1.csv"
df_updated.to_csv(updated_file_path, index=False)

# Display the updated dataset
print(df_updated)  # نمایش دیتافریم در کنسول

# Provide the path for further use
updated_file_path


       A      B      C       D       E       F       G       H       I       J
0      1  39353  85475  117961  118300  123472  117905  117906  290919  117908
1      1  17183   1540  117961  118343  123125  118536  118536  308574  118539
2      1  36724  14457  118219  118220  117884  117879  267952   19721  117880
3      1  36135   5396  117961  118343  119993  118321  240983  290919  118322
4      1  42680   5905  117929  117930  119569  119323  123932   19793  119325
...   ..    ...    ...     ...     ...     ...     ...     ...     ...     ...
32774  1  20364   2908  117961  118343  118979  120344  303717  118424  120346
32775  1  42093  52105  118595  118596   81476  121469  130834  118424  121471
32776  1  80473   7243  117961  118343  118609  120357  125779  118424  120359
32777  1  73547   4707  117961  118300  121108  119928  150810  118331  119929
32778  1  40230  46149  117961  118327    4674  118784  174486  290919  118786

[32779 rows x 10 columns]


'/content/updated_d1.csv'

In [20]:
batch_size = 8  # تعداد درخواست‌های لازم برای آپدیت مدل

if len(random_access_requests) >= batch_size:
    print("🚀 تعداد درخواست‌های جدید به حد لازم رسید. آموزش مجدد مدل آغاز می‌شود...")


🚀 تعداد درخواست‌های جدید به حد لازم رسید. آموزش مجدد مدل آغاز می‌شود...


In [30]:
# بارگذاری دیتاست جدید
import pandas as pd
# خواندن دیتاست
dataset = pd.read_csv("/content/updated_d1.csv")  # بدون فاصله اضافی
print(dataset.head())

X = dataset.drop(columns=['A']).values  # بدون فاصله اضافی
y = dataset['A'].values

    # نرمال‌سازی داده‌ها
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], 1, X.shape[1])
from tensorflow.keras.models import load_model
import numpy as np


    # بارگذاری مدل و آموزش مجدد
model = load_model("model.h5")
model.fit(X, y, epochs=2, batch_size=32, verbose=1)
model.save("model.h5")

print("✅ مدل به‌روزرسانی شد!")



   A      B      C       D       E       F       G       H       I       J
0  1  39353  85475  117961  118300  123472  117905  117906  290919  117908
1  1  17183   1540  117961  118343  123125  118536  118536  308574  118539
2  1  36724  14457  118219  118220  117884  117879  267952   19721  117880
3  1  36135   5396  117961  118343  119993  118321  240983  290919  118322
4  1  42680   5905  117929  117930  119569  119323  123932   19793  119325
Epoch 1/2


NotImplementedError: numpy() is only available when eager execution is enabled.

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import tensorflow as tf

# 🚀 1️⃣ تضمین اجرای Eager Execution
tf.config.run_functions_eagerly(True)

# 🚀 2️⃣ بررسی فایل دیتاست (مطمئن شوید که آپلود شده است)
file_path = "/content/updated_d1.csv"
if not os.path.exists(file_path):
    print("❌ خطا: فایل دیتاست یافت نشد! ابتدا دیتاست را آپلود کنید.")
    exit()

# 🚀 3️⃣ بارگذاری دیتاست
dataset = pd.read_csv(file_path)
print("✅ دیتاست با موفقیت بارگذاری شد.")

# 🚀 4️⃣ آماده‌سازی داده‌ها
X = dataset.drop(columns=['A']).values  # حذف ستون هدف
y = dataset['A'].values

# 🚀 5️⃣ نرمال‌سازی داده‌ها
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# 🚀 6️⃣ تبدیل `X` برای LSTM ([samples, time_steps, features])
X = X.reshape(X.shape[0], 1, X.shape[1])  # شکل مناسب برای LSTM

# 🚀 7️⃣ تبدیل `X` و `y` به NumPy برای جلوگیری از خطای `numpy()`
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# 🚀 8️⃣ بارگذاری مدل (بررسی وجود فایل مدل)
model_path = "model.h5"
if not os.path.exists(model_path):
    print("❌ خطا: فایل مدل پیدا نشد! ابتدا مدل را آموزش دهید و ذخیره کنید.")
    exit()

model = load_model(model_path)
print("✅ مدل با موفقیت بارگذاری شد.")

# 🚀 9️⃣ کامپایل مجدد مدل برای رفع هشدار
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 🚀 🔟 آموزش مجدد مدل با Mini-Batch Learning
model.fit(X, y, epochs=2, batch_size=32, verbose=1)

# 🚀 1️⃣1️⃣ ذخیره مدل به‌روزرسانی‌شده
model.save(model_path)
print("✅ مدل با موفقیت به‌روزرسانی شد و ذخیره گردید! 🎉")


✅ دیتاست با موفقیت بارگذاری شد.


✅ مدل با موفقیت بارگذاری شد.
Epoch 1/2


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1025/1025 ━━━━━━━━━━━━━━━━━━━━ 115s 112ms/step - accuracy: 0.9429 - loss: 0.2603
Epoch 2/2
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 111s 108ms/step - accuracy: 0.9423 - loss: 0.2233


✅ مدل با موفقیت به‌روزرسانی شد و ذخیره گردید! 🎉
